# REIT Stocks (Real Estate Investment Trust) Statistical analyis

REITS are an alternative investment option to those that want to invest in real estate, but do not have the funds to buy property or simply do not qualify for mortgage. Reits historically have delivered competitive high returns based on long term capital appreciation. This analysis will use hvplots visualizations to plot the top 15 REITS by market CAP, calculate rolling statistics, betas and compare them to the trends of S&P 500 market Cap index over time. 

In [6]:
# Import libraries and dependencies
import pandas as pd
from pathlib import Path
%matplotlib inline

In [7]:
# Import hvplot
import hvplot.pandas

### Prepare Data for visualization

In [9]:
# Set the path
file_path = Path('../../Resources/REITmarketcap.csv')

# Read in the CSV as a DataFrame
REIT_csv = pd.read_csv(file_path)

#Grab the symbol, volue and price per earning ratio
Volume_PE = REIT_csv.loc[:, ['Symbol','Volume', 'PE Ratio (TTM)']]

#Set the Symbol as index
Volume_PE.set_index(Volume_PE["Symbol"], inplace=True)

#Drop the extra symbol column
Volume_PE.drop(columns=["Symbol"], inplace=True)

# Grab the `Symbol` and `Market Cap` columns
market_cap = REIT_csv.loc[:, ['Symbol', 'Market Cap']]

# Set the 'Symbol' as the index
market_cap.set_index(market_cap['Symbol'], inplace=True)

# Drop the extra 'Symbol' column
market_cap.drop(columns=['Symbol'], inplace=True)

# Filter down to 15 companies with the largest market caps
top_15_market_cap = market_cap.nlargest(15,'Market Cap')

FileNotFoundError: [Errno 2] File ../../Resources/REITmarketcap.csv does not exist: '../../Resources/REITmarketcap.csv'

In [ ]:
REIT_csv.head()

### Plot top 5 REIT Stocks all variables

In [ ]:
# PLot TOP 5 REIT all variables
REIT_csv.set_index(REIT_csv["Symbol"], inplace=True)
REIT_csv.head(5).hvplot()

### Create line Plot of top 15 REITS by Market Cap

In [ ]:
# Generate standard hvplot to plot top 15 REITS by market cap
top_15_market_cap.hvplot(title='Top 15 Market Cap REIT Companies (in billions)', color= "blue")

### ### Create a bar Plot of top 15 REITS by Market Cap

In [ ]:
# Plot a hvplot bar chart of the top 15 market cap companies
top_15_market_cap.hvplot.bar(title='Top 15 Market Cap REIT Companies (in billions)')

In [ ]:
#Analyze the relationship between volume of transactions and price per earnings
Volume_PE.hvplot(kind= 'scatter', x='Volume', y='PE Ratio (TTM)', color= "red")

### Import and clean the data of 3 largest REIT stocks by Market Cap
## Set the file paths for the CSVs using the Path class from the pathlib library

Used adj. close price to factor dividends in the calculations, because by law REITs are required to pay out at tleast 90% of their taxable income


In [ ]:
file_path = Path('../../Resources/PLD.csv')

# Read in the CSV as a DataFrame
PLD_csv = pd.read_csv(file_path, index_col="Date", parse_dates=True, infer_datetime_format=True)

In [ ]:
file_path = Path('../../Resources/CCI.csv')

# Read in the CSV as a DataFrame
CCI_csv = pd.read_csv(file_path, index_col="Date", parse_dates=True, infer_datetime_format=True)

In [ ]:
file_path = Path('../../Resources/AMT.csv')

# Read in the CSV as a DataFrame
AMT_csv = pd.read_csv(file_path, index_col="Date", parse_dates=True, infer_datetime_format=True)

In [ ]:
combined_df = pd.concat([PLD_csv, CCI_csv, AMT_csv], axis="columns", join="inner")

# Sort datetime index in ascending order (past to present)
combined_df.sort_index(inplace=True)

# Display a few rows
combined_df.head()

In [ ]:
combined_df.columns = ['PLD', 'CCI', 'AMT']
combined_df.head()

In [ ]:
# Use the `pct_change` function to calculate daily returns of closing prices for each column
daily_returns = combined_df.pct_change().dropna()
daily_returns.head()

In [ ]:
daily_returns.hvplot()

In [ ]:
# Use the `dot` function to multiply the weights by each stock's daily return to get the portfolio daily return
weights = [0.3, 0.3, 0.3]

portfolio_returns = daily_returns.dot(weights)
portfolio_returns.head()

In [ ]:
# Use the `plot` function to plot the daily portfolio returns
portfolio_returns.hvplot()

In [ ]:
# Use the `cumprod` function to cumulatively multiply each element in the Series by it's preceding element until the end
cumulative_returns = (1 + portfolio_returns).cumprod() - 1
cumulative_returns.head()

In [ ]:
# Use the `plot` function to plot the cumulative portfolio returns
cumulative_returns.hvplot()

### Rolling statistics REITS Portfolio

In [ ]:
# Plot a rolling 7-day mean of REIT's closing prices
combined_df.rolling(window=7).mean().hvplot()

In [ ]:
# Plot a rolling 7-day std dev of REITS's closing prices
combined_df.rolling(window=7).std().hvplot()

In [ ]:
# Plot a rolling 30-day mean of MSFT's closing prices
combined_df.rolling(window=30).mean().hvplot.step()

In [ ]:
# Plot a rolling 30-day std dev of MSFT's closing prices
combined_df.rolling(window=30).std().hvplot()

### Read S$P 500 Data and calculate daily returns

In [ ]:
#Bring in S&P 500 Market Cap index
file_path = Path('../../Resources/SPindex.csv')

# Read in the CSV as a DataFrame
sp500_df = pd.read_csv(file_path, index_col="Date", parse_dates=True, infer_datetime_format=True)
sp500_df.head()

### Calculate S&P 500 Daily returns

In [ ]:
# Use the `pct_change` function to calculate daily returns of closing prices for each column
SP500_daily_returns = sp500_df.pct_change().dropna()
SP500_daily_returns.head()

In [ ]:
#Plot S&P 500 daily returns
SP500_daily_returns.hvplot()

### Combine REIT`S Stock with S&P 500 Returns

In [ ]:
# Create a new pivot table where the columns are the closing prices for each ticker
new_combined_df = pd.concat([PLD_csv, CCI_csv, AMT_csv, sp500_df], axis="columns", join="inner")

# Sort datetime index in ascending order (past to present)
new_combined_df.sort_index()

# Set column names to 'FB' 'TWTR', 'SNAP', and 'S&P 500'
new_combined_df.columns = ["PLD", "CCI", "AMT", "S&P 500"]

# Display a few rows
new_combined_df.head()

In [ ]:
#Calculate new daily returns
# Use the `pct_change` function to calculate daily returns of closing prices for each column
new_daily_returns = new_combined_df.pct_change().dropna()
new_daily_returns.head()

### Calculate Covariance of REITS Stock Returns vs. S&P 500 Returns

In [ ]:
# Calculate covariance of all daily returns of social media stocks vs. S&P 500
pld_covariance = new_daily_returns['PLD'].cov(new_daily_returns['S&P 500'])
cci_covariance = new_daily_returns['CCI'].cov(new_daily_returns['S&P 500'])
amt_covariance = new_daily_returns['AMT'].cov(new_daily_returns['S&P 500'])


### Calculate Variance of S&P 500 Returns

In [ ]:
# Calculate variance of all daily returns of social media stocks vs. S&P 500
variance = new_daily_returns['S&P 500'].var()
variance

#High variance in stocks are usually associated with high risk. 

### Calculate Beta Values of REIT Stocks

In [ ]:
# Calculate beta of all daily returns of social media stocks
pld_beta = pld_covariance / variance
cci_beta = cci_covariance / variance
amt_beta = amt_covariance / variance

print(f"PLD: {pld_beta} | CCI: {cci_beta} | AMT: {amt_beta}")

#Betas below 1.0 usially tell that the stocs are less volatile than the market

### Calculate 30-Day Rolling Betas of REIT Stock Returns vs. S&P 500 Returns

In [ ]:
# Calculate 30-day rolling covariance of REIT vs. S&P 500 and plot the data
rolling_pld_covariance = new_daily_returns['PLD'].rolling(window=30).cov(new_daily_returns['S&P 500'])
rolling_cci_covariance = new_daily_returns['CCI'].rolling(window=30).cov(new_daily_returns['S&P 500'])
rolling_amt_covariance = new_daily_returns['AMT'].rolling(window=30).cov(new_daily_returns['S&P 500'])

# Calculate 30-day rolling variance of S&P 500
rolling_variance = new_daily_returns['S&P 500'].rolling(window=30).var()

# Calculate 30-day rolling beta of REIT and plot the data
rolling_pld_beta = rolling_pld_covariance / rolling_variance
rolling_cci_beta = rolling_cci_covariance / rolling_variance
rolling_amt_beta = rolling_amt_covariance / rolling_variance
rolling_amt_beta

### Plot Rolling 30-Day Betas of REIT Stocks

In [ ]:
# Set the figure and plot the different datasets as multiple trends on the same figure
ax = rolling_pld_beta.plot(figsize=(20, 10), title='Rolling 30-Day Beta of REITS stocks')
rolling_cci_beta.plot(ax=ax)
rolling_amt_beta.plot(ax=ax)


# Set the legend of the figure
ax.legend(["PLD", "CCI", "AMT"])

 Based on the above calculation, we concluded that S&P 500 market cap index are riskier than the three REIT stocks, although the potential for higher returns is greater. Depending on risk tolerance, for risky averse investors, investing in REITS can be a better option, while those seeking higher return may opt to invest in S&P 500.